In [ ]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import yfinance as yf

def fetch_data_from_yfinance(ticker, period):
    data = yf.download(ticker, period=period, interval='1d')
    if data.empty:
        raise ValueError("Nie udało się pobrać danych z yfinance.")
    data = data[['Close']].rename(columns={'Close': 'price'})
    return data

ticker = "BTC-USD"
period = "5y"
data = fetch_data_from_yfinance(ticker, period)

if data.empty or 'price' not in data.columns:
    raise ValueError("Nie udało się pobrać danych lub kolumna 'price' jest niedostępna.")
data['price'].fillna(method='ffill', inplace=True)

scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data['price'].values.reshape(-1, 1))

def build_dataset(dataset, time_step=7):
    X, y = [], []
    for i in range(len(dataset) - time_step):
        X.append(dataset[i:i + time_step, 0])  
        y.append(dataset[i + time_step, 0])  
    return np.array(X), np.array(y)

time_step = 7
train_size = int(len(data_scaled) * 0.7)
train_data = data_scaled[:train_size]
test_data = data_scaled[train_size:]

X_train, y_train = build_dataset(train_data, time_step)
X_test, y_test = build_dataset(test_data, time_step)

    model = Sequential([
        Dense(128, activation='relu', input_shape=(time_step,)),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(1)  
    ])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

start_time = time.time()
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=64,
    verbose=1,
    callbacks=[early_stopping]
)
training_time = time.time() - start_time

plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Training Loss', color='blue')
plt.plot(history.history['val_loss'], label='Validation Loss', color='orange')
plt.title('MLP Loss Function During Training')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid()
plt.show()

start_time = time.time()
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)
inference_time = time.time() - start_time

train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)

plt.figure(figsize=(12, 6))
plt.plot(data.index, data['price'], label='Actual Data', color='blue')
plt.plot(data.index[time_step:len(train_predict) + time_step], train_predict.flatten(), label='Train Predictions', color='green', linestyle='dashed')
plt.plot(data.index[len(data_scaled) - len(test_predict):], test_predict.flatten(), label='Test Predictions', color='orange', linestyle='dashed')
plt.title('Historical Data and Test Predictions (MLP)')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.grid()
plt.show()

y_train_actual = scaler.inverse_transform(y_train.reshape(-1, 1))
y_test_actual = scaler.inverse_transform(y_test.reshape(-1, 1))

mae_train = mean_absolute_error(y_train_actual, train_predict)
mae_test = mean_absolute_error(y_test_actual, test_predict)

mse_train = mean_squared_error(y_train_actual, train_predict)
mse_test = mean_squared_error(y_test_actual, test_predict)

rmse_train = np.sqrt(mse_train)
rmse_test = np.sqrt(mse_test)

r2_train = r2_score(y_train_actual, train_predict)
r2_test = r2_score(y_test_actual, test_predict)

print('\n Metryki czasowe:')
print(f'Czas treningu: {training_time:.2f} sekund')
print(f'Czas inferencji: {inference_time:.2f} sekund')

print("Metryki skuteczności modelu MLP:")
print(f"MAE (Train): {mae_train:.4f}, MAE (Test): {mae_test:.4f}")
print(f"MSE (Train): {mse_train:.4f}, MSE (Test): {mse_test:.4f}")
print(f"RMSE (Train): {rmse_train:.4f}, RMSE (Test): {rmse_test:.4f}")
print(f"R² (Train): {r2_train:.4f}, R² (Test): {r2_test:.4f}")

num_prediction_days = 7
last_sequence = X_test[-1]
future_predictions = []

for _ in range(num_prediction_days):
    next_prediction = model.predict(last_sequence.reshape(1, time_step))[0, 0]
    future_predictions.append(next_prediction)
    last_sequence = np.roll(last_sequence, -1)
    last_sequence[-1] = next_prediction

future_predictions = scaler.inverse_transform(np.array(future_predictions).reshape(-1, 1))
future_dates = pd.date_range(start=data.index[-1] + pd.Timedelta(days=1), periods=num_prediction_days, freq='D')

plt.figure(figsize=(12, 6))
plt.plot(data.index, data['price'], label='Actual Data', color='blue')
plt.axvline(x=data.index[-1], color='black', linestyle='--', label='Prediction Start')
plt.plot(future_dates, future_predictions.flatten(), label='Future Predictions', color='red', linestyle='dotted')
plt.title('Historical Data and Future Predictions (MLP)')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.grid()
plt.show()


In [ ]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import yfinance as yf

def fetch_data_from_yfinance(ticker, period):
    data = yf.download(ticker, period=period, interval='1d')
    if data.empty:
        raise ValueError("Nie udało się pobrać danych z yfinance.")
    data = data[['Open', 'High', 'Low', 'Close']]
    data.fillna(method='ffill', inplace=True)  
    return data

ticker = "BTC-USD"
period = "5y"
data = fetch_data_from_yfinance(ticker, period)

scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data)

def build_dataset(dataset, time_step=7):
    X, y = [], []
    for i in range(len(dataset) - time_step):
        X.append(dataset[i:i + time_step, :])  
        y.append(dataset[i + time_step, -1])  
    return np.array(X), np.array(y)

time_step = 7
train_size = int(len(data_scaled) * 0.7)
train_data = data_scaled[:train_size]
test_data = data_scaled[train_size:]

X_train, y_train = build_dataset(train_data, time_step)
X_test, y_test = build_dataset(test_data, time_step)

model = Sequential([
    Dense(128, activation='relu', input_shape=(time_step, 4)),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(1)  
])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

start_time = time.time()
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=200,  
    batch_size=64,
    verbose=1,
    callbacks=[early_stopping]
)
training_time = time.time() - start_time

plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Training Loss', color='blue')
plt.plot(history.history['val_loss'], label='Validation Loss', color='orange')
plt.title('MLP Loss Function During Training')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid()
plt.show()

start_time = time.time()
train_predict = model.predict(X_train).reshape(-1, 1)
test_predict = model.predict(X_test).reshape(-1, 1)
inference_time = time.time() - start_time

train_predict_padded = np.concatenate((np.zeros((len(train_predict), 3)), train_predict), axis=1)
test_predict_padded = np.concatenate((np.zeros((len(test_predict), 3)), test_predict), axis=1)

train_predict = scaler.inverse_transform(train_predict_padded)[:, -1]
test_predict = scaler.inverse_transform(test_predict_padded)[:, -1]

y_train_actual_padded = np.concatenate((np.zeros((len(y_train), 3)), y_train.reshape(-1, 1)), axis=1)
y_test_actual_padded = np.concatenate((np.zeros((len(y_test), 3)), y_test.reshape(-1, 1)), axis=1)

y_train_actual = scaler.inverse_transform(y_train_actual_padded)[:, -1]
y_test_actual = scaler.inverse_transform(y_test_actual_padded)[:, -1]

min_len_train = min(len(y_train_actual), len(train_predict))
min_len_test = min(len(y_test_actual), len(test_predict))

y_train_actual = y_train_actual[:min_len_train]
train_predict = train_predict[:min_len_train]

y_test_actual = y_test_actual[:min_len_test]
test_predict = test_predict[:min_len_test]

train_index = data.index[time_step: time_step + len(train_predict)]
test_index = data.index[train_size + time_step: train_size + time_step + len(test_predict)]

train_predict = train_predict[:len(train_index)]
test_predict = test_predict[:len(test_index)]

plt.figure(figsize=(12, 6))

plt.plot(data.index, data['Close'], label='Actual Data', color='blue')

plt.plot(train_index, train_predict, label='Train Predictions', color='green', linestyle='dashed')

plt.plot(test_index, test_predict, label='Test Predictions', color='orange', linestyle='dashed')

plt.title('Historical Data and Test Predictions (MLP)')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.grid()
plt.show()

mae_train = mean_absolute_error(y_train_actual, train_predict)
mae_test = mean_absolute_error(y_test_actual, test_predict)

mse_train = mean_squared_error(y_train_actual, train_predict)
mse_test = mean_squared_error(y_test_actual, test_predict)

rmse_train = np.sqrt(mse_train)
rmse_test = np.sqrt(mse_test)

r2_train = r2_score(y_train_actual, train_predict)
r2_test = r2_score(y_test_actual, test_predict)

print('\nMetryki czasowe:')
print(f'Czas treningu: {training_time:.2f} sekund')
print(f'Czas inferencji: {inference_time:.2f} sekund')

print("\nMetryki skuteczności modelu MLP (Wariant 2 - OHLC):")
print(f"MAE (Train): {mae_train:.4f}, MAE (Test): {mae_test:.4f}")
print(f"MSE (Train): {mse_train:.4f}, MSE (Test): {mse_test:.4f}")
print(f"RMSE (Train): {rmse_train:.4f}, RMSE (Test): {rmse_test:.4f}")
print(f"R² (Train): {r2_train:.4f}, R² (Test): {r2_test:.4f}")
